# Wrangling OpenStreetMap Data for SW West Virginia

## Motivation
In order to develop my data wrangling skills, I am auditing and organizing the OpenStreetMap data for SW West Virginia. I chose West Virginia for multiple reasons:
1. My wife's family is from there (and most still live in the state) 
2. I assumed that (like with most self-reported or survey-based data collection efforts), a state with a high proportion of poor and rural areas is likely to have very poor data coverage and quality. This is something that my work on map data may help
3. The state of WV is an area racked by a number of unfortunate statistics, not the least of which is [a very high drug overdose rate](https://www.cdc.gov/drugoverdose/data/statedeaths.html). This area of West Virginia (in particular, Huntington, WV) [suffers particularly badly](https://www.npr.org/2017/06/29/534868012/what-happens-when-the-heroin-epidemic-hits-small-town-america), with the city of Huntington sometimes being called the drug overdose death capital of America.

In wrangling the data for this region of the US, I hope to provide some value to an otherwise ignored set of communities. It is my hope that I will be able to take my progress here and push the audited data to OpenStreetMap as a final step.

## Data Provenance

First of all, let's establish where these data came from and some basic information about them. The data were pulled as a custom extract from https://mapzen.com/ and the final file size for the region (unzipped) is 538 MB. An image of the region extracted is shown here **(note that this region includes WV as well as some portions of VA, KY, and OH)**: ![](imgs/SW_WV_ExtractConfirm.png).

## Project Steps

I'm going to tackle this project using the following steps (mostly included here for my own mental organization, but hopefully also helpful for anyone following this work too!):

1. Sample the data to generate a relatively small data set (e.g. 1-10 MB) that I can sift through to identify recurring concerns as part of my data audit. The code I will use for sampling the data is called *SampleMapData_Small.py*. 

2. Audit the data sample in the tradition of the Udacity Data Wrangling course, using NumPy and Pandas in Python. This will entail:
    1. **Auditing data validity:** do the data conform to a pre-defined data schema? In the case of this project, a relevant question would be *Do the tags include zip codes that are within the region of interest?*
    2. **Auditing data accuracy:** do the data conform to some gold standard? An example of this will likely be zip codes only being 5 or 9 digits long.
    3. **Auditing data completeness:** one approach I may take to this is ensuring that cities or counties I know should be included in the region I'm parsing are actually recorded in the data. Given the region chosen here, one obvious audit that will need to occur is identifying easily the non-WV data in the set. While these data are not inherently invalid, different state policies regarding the naming of roads, counties, etc. may have an impact on the seeming validity of the data at hand and should be identified early.
    4. **Auditing data consistency:** I will investigate this by looking across tags of the same type to ensure a standard format is being followed throughout and, if it is not, determining if a correction is needed to answer the questions of greatest interest to me. I'll also investigate other issues of internal consistency as they arise.
    5. **Auditing data uniformity:** for these data, this will likely take the form of checking whether data values are within a reasonable range (e.g. no latitudes or longitudes well outside the region of interest).

3. Check to see if more audit problems are found when sampling a larger data set than previously. Keep iterating on this approach of "increase sample size, sample, audit" until no new audit failures are found in the full data set.

4. Correct the data problems identified in the audits as part of the CSV data export process that follows the data schema provided by the Udacity team. This schema creates tables tracking `nodes`, `node_tags`, `ways`, `ways_tags`, and `ways_nodes` and allows for the creation of the SQL database needed for recording these data.

5. Create a SQL database with the provided schema.

5. Import the CSV data files (one per SQL table) into the SQL database created.

6. Query this database in a variety of ways to:
    1. Check to make sure no other auditing errors exist. If any are found, deal with them by modifying the auditing code, re-creating the CSV files, and removing then re-adding records in the SQL database (if necessary).
    2. Determine some descriptive aspects of the data. For example, data-oriented queries of interest could include the number of users contributing and identifying some representative contributions of the highest-frequency contributors. This could identify any recurring issues with the data these individuals are supplying and speed up any further auditing that may be needed. Region-specific auditing would include things like determining if the number of cities and counties are accurate. Statistics specifically mentioned in the project rubric (with some additions from me) are:
        1. Size of all files used in this project
        2. Number of unique users
        3. Number of nodes and ways
        5. Numbers of nodes lacking any child tags to describe them beyond their latitude/longitude
    
    3. Investigate any other interesting counts present in the data, such as:
        1. The number of unincorporated townships vs. the number of cities/villages
        2. Comparing the number of residential properties to the number of commercial properties.         
        3. WV is known by many to have issues with the presence (or lack thereof) of basic infrastructure needs, given its high density of rural locations. As such, I'll also explore how the count of residences compares to the count of:
            * grocery stores, 
            * restaurants, 
            * alcohol serving-selling businesses, and 
            * hospitals + doctor offices 
        
        This will potentially illuminate the issue of "healthcare and food deserts." The healthcare component in particular is relevant to the aforementioned overdose concerns for this area. I include alcohol-oriented establishments in these results to replicate [a similar study done in 2013](https://www.usatoday.com/story/dispatches/2013/12/06/top-bar-and-pizza-cities/3882089/) regarding the number of bars per capita vs. food options per capita. In my case, I plan to compare the different counties in terms of their ratios of alcohol to food, alcohol to healthcare, and fast food to healthcare, among other combinations.

#### Note

As all of the significant problems seem to exist in nodes and ways with child tags, I modified my sampling algorithm (when using smaller sample OSM files for initial code development) to skip any nodes or ways that were not parents of tags, so as to make for easier data parsing and spot analysis. Doing so reduced the sampled file (with sample frequency = 1000 tags) from 6,832 lines to 4,582 lines.

# Auditing the Data
### Issues Requiring Correction Identified During Audit

3. Zip code formatting and/or just plain wrong-ness of zip code.
4. State/county inconsistency, formatting, or representation as a code number instead of a name.
4. Amenity/shop types were (rarely) incorrectly labeled/spelled. 

The items mentioned here will be explored in more detail later in this report.

### Non-issues that Surprised Me (non-issues in the sense that they do not require correction/cannot reasonably be corrected for in this project)

3. **No latitudes or longitudes were identified that are outside the expected bounds.**
    * These bounds (four corners of a rectangle) are: 
        * [-82.6730344023,37.1523636424],
        * [-80.2011105742,37.1523636424],
        * [-80.2011105742,39.0498347562],
        * [-82.6730344023,39.0498347562]
        * **Basic rule: longitude should be between -82.67 and -80.20, latitude should be between 37.15 and 39.05**
6. **Multiple businesses were identified that did not have any business type (e.g. shop, amenity, etc.) associated with them.** *At this time, there is no obvious way to correct for this error, as no discernible pattern exists as to when the tag is included or excluded.* For example, many of the nodes that were clearly businesses (based upon their names) had `k="name"` values, but other nodes also had this and were just bus stops, waterways, or something else entirely. As a result, this will be an issue that needs to remain for now, until a gold standard data set for business names can be identified and used in the auditing process (e.g. by comparing `k="name"` values to the list of known businesses and extracting the business type label from that same list to put into the OSM data).


# Addressing the Issues that Require Data Corrections

### Formatting of Zip Codes

**Some issues identified in this area:**
1. There is one postcode/zip code that is just plain wrong, as its value is 'WV'
2. Some of the codes appear to follow the 9-digit zip code standard instead of the more general 5-digit standard (e.g. '12345-1111')
3. Some of the codes include lists of zip codes separated by either colons or semi-colons

**I will correct these issues during export into CSV files for the SQL tables by (these are referenced by preceding issue numbers, resp.):**
1. This is a node that represents a house in Charleston, WV on Upper Ridgeway Road. Given that [this is a fairly short road](https://www.google.com/maps/place/Upper+Ridgeway+Rd,+Charleston,+WV+25314/@38.3365169,-81.6423879,17z/data=!3m1!4b1!4m5!3m4!1s0x884f2cdc4bbe7763:0x80e8115b09ccb392!8m2!3d38.3365169!4d-81.6401992) unlikely to have more than one zip code associated with it, I will assign it the zip found using Google Maps (25314) when extracting this node to CSV format (node ID = 2625119248)
2. Shortening each 9-digit zip code to only include the first 5 digits
3. Extracting individual zip codes when they appear as lists, giving them each their own tag record associated with the node/way tag ID in question. This isn't a perfect solution, as the ones with colons (e.g. 11111:22222) are clearly meant to indicate a range of zip codes, but without a good bit more GIS-based calculation, it's impossible to tell what that range is meant to be. As such, I'll simply include the beginning and end zips of that range as individual records.

NOTE: Each zip code will be encoded into the nodes_tags or ways_tags table as `addr:postcode` as this is the most common way to record a zip code in OSM.

### State/County Naming 
            
**County and state names are inconsistent**, depending upon which system generated them (e.g. GNIS, Tiger, etc.). This is important, as so many features of WV are referenced by county by WV residents, thus any analysis will need to be based off of counties to have any intuitive value to residents (and likely the same is true of state and local legislators). Also, if there are states being recorded that are clearly not supposed to be within the territory sampled, we need to correct that.

**Some issues identified in this area:**
1. The state name is included with the county name (e.g. `tiger:county` = "County, State (2-letter)") and not as a separate data field
2. Some (ways, presumably) also include *lists* of county,state entries in a single tag, separated by colons or semicolons (sometimes with, sometimes without, leading and trailing spaces for each item in the list).
3. The county names are referenced by differing tag keys across the dataset (e.g. `gnis:county_name` + `addr:state`; OR `gnis:County` + `gnis:ST_alpha`; and many others)
4. Formatting is inconsistent for the state names, with the first letter sometimes being capitalized (e.g. 'Wv'), both letters being capitalization (e.g. 'WV'), neither being capitalized (e.g. 'wv'), or the state's full name being utilized (e.g. 'West Virginia').
5. Strangely, "CA" is listed as one of the states found in this OSM file. That is clearly wrong.
6. The tags only include a numeric `gnis:county_id` and `gnis:state_id` for some nodes and ways, typically amenities and shops (e.g. instead of naming the county/state as they do for non-commercial nodes). 
    1. These commercial locations also typically lack street addresses, only including the requisite longitude and latitude. 


**I will correct these issues during export into CSV files for the SQL tables by (these are referenced by preceding issue numbers, resp.):**
1. Removing the state from any county entry, making a state-specific tag record for the state name extracted (when exporting to CSV) and keeping the county name as its own data field (these would be of the form `addr:county` or `addr:state` in the nodes_tags/ways_tags table)
2.  Extracting each state/county name from the list and storing in a separate tag record for the node/way in question
3. Auditing done by `Audit_Simple.py` has revealed that the following state and county tags are most relevant to our analysis here, and these will be the ones that are the focus of extraction:
    * **Types of County Tags**:
        * `gnis:County`
        * `gnis:County_num`
        * `gnis:county_id`
        * `gnis:county_name`
        * `is_in:county`
        * `tiger:county`

    * **Types of State Tags**
        * `addr:state`
        * `gnis:ST_alpha`
        * `gnis:ST_num`
        * `gnis:state_id`
        * `nist:state_fips`
4. The standard we will be using here will be two-letter state names with both letters capitalized.
5. Further analysis of the OSM file shows that this is just one node and that it relates to a school in Shady Spring, WV, not Shady Spring, CA. As such, this node (ID 398603731) has been flagged as one whose tag key `addr:state` needs to have its value changed from 'CA' to 'WV'.
6. I can create consistency by developing a mapping algorithm for county/state ID numbers to county and state names, using a download of historic US Census Bureau FIPS codes to ensure that the proper mapping and vintage of data set are being utilized. Counties will be recorded as `addr:county` and states will be `addr:state`.
    1. The problem of commercial locations lacking street addresses is beyond the scope of this project, as it requires substantial additional GIS data that is supposed to be provided by OSM in the first place.
    
NOTE: the 2007 import of USGS GNIS data into OpenStreetMaps had the fields `gnis:County_num` and `gnis:ST_num` which are the county and state FIPS codes, resp. The 2009 import of USGS GNIS data had similar fields, but with slightly different tag keys. These were `gnis:county_id` and `gnis:state_id`, which are also FIPS codes. **As such, my extraction of these data will include mapping these numbers to their actual names, as per these FIPS code mappings, using data downloaded from the US Census Bureau for FIPS codes.** [Please see here](https://wiki.openstreetmap.org/wiki/USGS_GNIS#Other_Tags) for the reference regarding these import actions for OSM and [here](https://www.census.gov/geo/reference/codes/cou.html) for the 2010 FIPS code data used in this project.

### Shop/Amenity Tag Formatting

There were no extraneous or misspelled amenity types/values as far as I could tell, except for the incorrect `amenity:'ATV Trails'` entry and the incorrectly-capitalized `shop:Tiles`. 

**I will correct this by changing these tags to `amenity:atv` and `shop:tiles`, resp., when exporting to CSV, even though these are not critical to my proposed analysis.**

## Populating Our SQL Database

At this point, using the code provided in the Helper Code directory, I have audited, corrected, and exported to CSV files all the data needed to populate my database. First, I'm going to set the database tables' schema using the schema provided by Udacity (data_wrangling_schema.sql). I'll accomplish this by running the following code in the terminal to create the new database, then set the schema, then import the data into the relevant tables:

`sqlite3 SW_WV_OSM.db
.read data_wrangling_schema.sql
.mode csv
.import "CSV for SQL Tables/nodes.csv" nodes
.import "CSV for SQL Tables/nodes_tags.csv" nodes_tags
.import "CSV for SQL Tables/ways.csv" ways
.import "CSV for SQL Tables/ways_tags.csv" ways_tags
.import "CSV for SQL Tables/ways_nodes.csv" ways_nodes`

## Time to Answer Our Research Questions!
Here, we discuss the different questions outlined earlier this report, using the results from a set of queries of our freshly-corrected map data, stored in an SQLite3 database. Queries here were carried out through the Python module DBQuerying.py in the Helper Code directory, but they could have just as easily been done in the sqlite3 terminal environment. However, I wanted to use the Python environment for this (using the DB-API database library standard) as I expect in the future querying of databases and manipulation of query results to be easier when done using pandas.

Let's dig in.

In [59]:
import sys
sys.path.append('HelperCode')

import DBQuerying as db

**First, let's check to see if our corrections addressed our auditing concerns. We'll address these in the same order that we tackled them earlier when discussing what our auditing found.** Querying to check if any of the previously-identified erroneous values still exist is tedious and not interesting code (and therefore violated the "succinctness" criterion of the review rubric for this project), so I just provide here an example query used for checking for the presence of semi-colons and colons in zip codes within both the ways_tags and nodes_tags tables:

```sql
SELECT COUNT(*)
FROM (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags)
WHERE key = 'postcode' and (value LIKE '%:%' or value LIKE '%;%');
```

**As it turns out, from running this query and the other audit queries like it, we have addressed all of our previous auditing concerns in this data set!** Now, on to more interesting questions.

## Size of all files used in this project

File | Size
--- | ---
*SW_WestVirginia.osm* | 538 MB
*SW_WV_OSM.db* | 296 MB
*nodes.csv* | 212.5 MB
*nodes_tags.csv* | 7 MB
*ways.csv* | 10 MB
*ways_tags.csv* | 38 MB
*ways_nodes.csv* | 61 MB

## Number of unique OpenStreetMap contributing users

In [186]:
query_list = ["SELECT COUNT(DISTINCT uid)",
              "FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)"]

db.run_query(query_list, "SW_WV_OSM.db")

,COUNT(DISTINCT uid)
0,1033


## Number of nodes

In [187]:
query_list = ["SELECT COUNT(DISTINCT id)",
              "FROM nodes"]

db.run_query(query_list, "SW_WV_OSM.db")

,COUNT(DISTINCT id)
0,2553791


## Number of ways

In [188]:
query_list = ["SELECT COUNT(DISTINCT id)",
              "FROM ways"]

db.run_query(query_list, "SW_WV_OSM.db")

,COUNT(DISTINCT id)
0,171207


## Numbers of nodes lacking any child tags to describe them beyond their latitude/longitude

As a LEFT JOIN on the `id` column between `nodes` and `nodes_tags` (in that order) would retain all records from `nodes`, even the ones that don't have a corresponding entry in `nodes_tags`, this is the best way to count how many there are. As it turns out, the vast majority of nodes (about 98%) don't have any child tags describing them.

In [189]:
query_list = ["SELECT COUNT(*)",
              "FROM nodes LEFT JOIN nodes_tags",
             "ON nodes.id = nodes_tags.id",
              "WHERE key IS NULL"]

db.run_query(query_list, "SW_WV_OSM.db")

,COUNT(*)
0,2511469


## Additional Investigations: Unincorporated Townships vs. Cities/Villages

West Virginia, unlike some other states, does not require all land within it to be incorporated, with a council or other governing body in charge of it. I suspect that, in fact, a large number of these unincorporated areas exist when compared to the more standard incorporated regions, based upon my experience in the state and conversations with my in-laws who reside there.

We can assume that (for nodes) `place:town` (e.g. `<tag k="place" v="town" />`), `place:village`, `place:hamlet`, and `place:isolated_dwelling` **almost always refer to unincorporated communities** (reference justifying this [can be found here](https://wiki.openstreetmap.org/wiki/United_States_admin_level#Unincorporated_areas)) for purposes of analysis. The only named `place` that is considerd always incorporated, in fact is, `place:city`, so we'll use that as our comparator.

In [190]:
query_list = ["SELECT value, COUNT(*) as Num",
              "FROM nodes_tags",
             "WHERE key = 'place'",
             "GROUP BY value",
             "ORDER BY Num DESC"]

db.run_query(query_list, "SW_WV_OSM.db")

,value,Num
0,hamlet,2776
1,village,134
2,locality,68
3,county,36
4,island,14
5,town,11
6,city,3
7,neighbourhood,3
8,farm,1
9,islet,1


It looks like we're dealing with a very rural area here, with lots of unincorporated areas. No real surprise there, given that no cities in West Virginia are particularly populous. Still, only 3 cities/places that would be considered incorporated? That's astounding. Which cities are we talking about here?

In [191]:
query_list = ["SELECT nodes_tags.value as City_Name",
              "FROM nodes_tags",
              "JOIN (SELECT id FROM nodes_tags WHERE key = 'place' and value = 'city') as a",
             "ON nodes_tags.id = a.id",
             "WHERE key = 'name'"]

db.run_query(query_list, "SW_WV_OSM.db")

,City_Name
0,Ashland
1,Charleston
2,Huntington


These results reveal that this approach to identifying incorporated areas in WV is a reasonable one, but not without its flaws. A quick Google search for Ashland, WV reveals that it is considered an unincorporated community, but Huntington and Charleston do fulfill our criteria. However, this is likely more an imperfect measure than not, as the likelihood of county seats being unincorporated is small. How many counties are we looking at, anyhow?

In [192]:
query_list = ["SELECT COUNT(DISTINCT(value)) as Num_Counties",
              "FROM nodes_tags",
              "WHERE key = 'county'"]

db.run_query(query_list, "SW_WV_OSM.db")

,Num_Counties
0,64


Yeah...it looks like my method for counting unincorporated areas could use some work. There's no way that there's 64 unique counties in our data set and only 3 places that would be considered incorporated. Perhaps the flaw is in only considering nodes? Let's see what the breakdown looks like for ways.

In [193]:
query_list = ["SELECT value, COUNT(*) as Num",
              "FROM ways_tags",
             "WHERE key = 'place'",
             "GROUP BY value",
             "ORDER BY Num DESC"]

db.run_query(query_list, "SW_WV_OSM.db")

,value,Num
0,neighbourhood,8
1,island,6
2,town,6
3,hamlet,4
4,Blacksburg,2
5,islet,2


Well that doesn't explain it. For future investigations, I could do generate the list of county seats (generated from an external source, like Wikipedia) from the list of counties present in this data set and do some spot checks to ensure that my logic of "county seats are always incorporated" is generally accurate.

## Additional Investigations: Comparing Number of Residential vs. Commercial Properties

Here we'll investigate how many nodes are considered residential vs. commercial. This is more to get a flavor of the region than anything else: does it have a lot of homes, a prevalence of industrial entities, or an even mixture? Presumably one wouldn't expect nearly the number of commercial properties as residential ones, as those businesses should theoretically have economies of scale that don't require one business property per consumer household...right?



**The best way to identify the quantity of residential vs. commercial use areas is by using the tag keys `k="landuse"` and `k="building"`.** The relevant options are as follows:
    * Residential
        * `landuse:residential`
        * `landuse:village_green`
        * `landuse:recreation_ground`
        * `landuse:allotments`
        * `building:apartments`
        * `building:farm` 
        * `building:house`
        * `building:detached`
        * `building:residential`
        * `building:dormitory`
        * `building:houseboat`
        * `building:bungalow`
        * `building:static_caravan`
        * `building:cabin`
    * Commercial
        * `landuse:commercial`
        * `landuse:depot`
        * `landuse:industrial`
        * `landuse:landfill`
        * `landuse:orchard`
        * `landuse:plant_nursery`
        * `landuse:port`
        * `landuse:quarry`
        * `landuse:retail`   
        * `building:hotel`
        * `building:commercial`
        * `building:industrial`
        * `building:retail`
        * `building:warehouse`
        * `building:kiosk`
        * `building:hospital`
        * `building:stadium`
**Note 1:** any types that appear to be mixed residential + commercial usage or have mixed ownership models (e.g. `landuse:farmyard` or `building:university`) have been excluded from consideration for the sake of clarity.

**Note 2:** [According to OSM](https://wiki.openstreetmap.org/wiki/Key:building) `building:farm` is a purely residential designation

**Note 3:** `building:static caravan` refers to a mobile home (semi)permanently left on a single site       

First, let's see what all kinds of landuse and building types we have to work with. No sense in writing a `LIKE` query to account for all of these if they make up the vast majority of nodes anyhow!

In [194]:
query_list = ["SELECT value, COUNT(*) as Num",
              "FROM (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags)",
             "WHERE key = 'landuse'",
             "GROUP BY value",
             "ORDER BY Num DESC",
             ]

db.run_query(query_list, "SW_WV_OSM.db")

,value,Num
0,cemetery,194
1,residential,171
2,reservoir,169
3,grass,133
4,forest,62
5,industrial,52
6,quarry,52
7,farmyard,47
8,retail,40
9,meadow,39


Hmmm...this is going to get complicated quickly as an SQL query, given the scope of landuse and building types that are considered residential vs. commercial. Let's use the DataFrame functionality I built into `db.run_query()` to make this assessment a little simpler.

In [195]:
residential_landuse_values = ['residential', 'village_green', 'recreation_ground', 'allotments']

residential_building_values = ['apartments', 'farm', 'house', 'detached', 'residential', 'dormitory', 'houseboat',
                              'bungalow', 'static_caravan', 'cabin']


commercial_landuse_values = ['commercial', 'depot', 'industrial', 'landfill', 'orchard', 'plant_nursery', 'port',
                            'quarry', 'retail']

commercial_building_values = ['hotel', 'commercial', 'industrial', 'retail', 'warehouse', 'kiosk', 'hospital',
                             'stadium']


landuse_query = ["SELECT value, COUNT(*) as Num",
              "FROM (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags)",
             "WHERE key = 'landuse'",
             "GROUP BY value",
             "ORDER BY Num DESC"]

building_query = ["SELECT value, COUNT(*) as Num",
              "FROM (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags)",
             "WHERE key = 'building'",
             "GROUP BY value",
             "ORDER BY Num DESC"]

landuse_df = db.run_query(landuse_query, "SW_WV_OSM.db")
building_df = db.run_query(building_query, "SW_WV_OSM.db")


In [196]:
building_df

,value,Num
0,yes,23756
1,detached,4268
2,house,1100
3,shed,674
4,apartments,515
5,industrial,375
6,garage,230
7,roof,226
8,commercial,162
9,university,136


In [197]:
residential_df = landuse_df[landuse_df['value'].isin(residential_landuse_values)]
residential_df = residential_df.append(building_df[building_df['value'].isin(residential_building_values)], 
                                       ignore_index=True)

residential_df

,value,Num
0,residential,171
1,recreation_ground,4
2,detached,4268
3,house,1100
4,apartments,515
5,residential,115
6,dormitory,28
7,static_caravan,21
8,cabin,1


In [146]:
commercial_df = landuse_df[landuse_df['value'].isin(commercial_landuse_values)]
commercial_df = commercial_df.append(building_df[building_df['value'].isin(commercial_building_values)], ignore_index=True)

commercial_df

,value,Num
0,industrial,52
1,quarry,52
2,retail,40
3,commercial,35
4,landfill,3
5,orchard,3
6,plant_nursery,1
7,industrial,375
8,commercial,162
9,retail,59


**OK, so: how many residential locations are there compared to commercial location?**

In [208]:
residential_sum = residential_df['Num'].sum()
commercial_sum = commercial_df['Num'].sum()

print("Residential count = {}".format(residential_sum))
print("Commercial count = {}".format(commercial_sum))

Residential count = 6223
Commercial count = 820


**As hypothesized, the residential counts far outnumber the commercial ones.**

## Additional Investigations: Food and Healthcare Deserts

Now, let's investigate the amount of food- and healthcare-related amenities as a function of county in our data set. 

WV is known by many to have issues with the presence (or lack thereof) of basic infrastructure needs, given its high density of rural locations. As such, I'll also explore how the count of residences compares to the count of:
            * food-selling businesses 
            * alcohol-serving/selling businesses, and 
            * hospitals + doctor offices 
        
This will potentially illuminate the issue of "healthcare and food deserts." The healthcare component in particular is relevant to the aforementioned overdose concerns for this area. I include alcohol-oriented establishments in these results to replicate [a similar study done in 2013](https://www.usatoday.com/story/dispatches/2013/12/06/top-bar-and-pizza-cities/3882089/) regarding the number of bars per capita vs. food options per capita. In my case, I plan to compare the different amenities to the number of residential properties identified previously.

**A variety of amenity types exist (in general, not necessarily in my data file, but often in the data file too) that satisfy my research questions** regarding food and healthcare deserts:
    * Grocery stores
        * `shop:grocery`
        * `shop:greengrocer`
        * `shop:convenience`
        * `shop:supermarket`
    * Restaurants
        * `amenity:restaurant`
        * `amenity:cafe`
        * `amenity:fast_food`
        * `amenity:food_court`
    * Alcohol serving/selling locations
        * `amenity:biergarten`
        * `amenity:pub`
        * `amenity:bar`
        * `shop:alcohol`
        * `shop:wine`
    * Healthcare locations
        * `amenity:clinic`
        * `amenity:doctors`
        * `shop:optician`
        * `amenity:dentist`
        * `amenity:hospital`
        * `healthcare:*`
        
**So first things first: let's figure out what we're working with for each of these categories!**

In [203]:
amenity_shop_query = ["SELECT value, COUNT(*) as Num",
                      "FROM (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags)",
                      "WHERE key = 'amenity' or key = 'shop'",
                      "GROUP BY value",
                      "ORDER BY Num DESC"]

healthcare_query = ["SELECT value, COUNT(*) as Num",
                      "FROM (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags)",
                      "WHERE key = 'healthcare'",
                      "GROUP BY value",
                      "ORDER BY Num DESC"]

amenity_shop_df = db.run_query(amenity_shop_query, "SW_WV_OSM.db")
healthcare_df = db.run_query(healthcare_query, "SW_WV_OSM.db")


amenity_shop_df

,value,Num
0,place_of_worship,2840
1,grave_yard,2736
2,school,2423
3,parking,1773
4,post_office,735
5,restaurant,336
6,fast_food,275
7,fuel,271
8,bench,216
9,recycling,174


**That's a lot of places of worship and graveyards!**

OK, so let's now filter locations by our pre-defined lists of types. Note that I'm not going to concern myself with manipulating `healthcare_df` as it is an empty DataFrame, indicating a lack of those keys in this data set.

In [209]:
food = ['greengrocer', 'convenience', 'supermarket', 'restaurant', 'cafe', 'fast_food', 'food_court']
alcohol = ['biergarten', 'pub', 'bar', 'alcohol', 'wine']
healthcare = ['doctors', 'optician', 'dentist', 'hospital']

food_df = amenity_shop_df[amenity_shop_df['value'].isin(food)]
alcohol_df = amenity_shop_df[amenity_shop_df['value'].isin(alcohol)]
healthcare_df = amenity_shop_df[amenity_shop_df['value'].isin(healthcare)]

food_sum = food_df['Num'].sum()
alcohol_sum = alcohol_df['Num'].sum()
healthcare_sum = healthcare_df['Num'].sum()

print("Number of food-oriented establishments = {}".format(food_sum))
print("Number of alcohol-oriented establishments = {}".format(alcohol_sum))
print("Number of firms specializing in food and/or drink = {}".format(alcohol_sum + food_sum))


print("\nNumber of healthcare-oriented establishments = {}".format(healthcare_sum))

Number of food-oriented establishments = 894
Number of alcohol-oriented establishments = 59
Number of firms specializing in food and/or drink = 953

Number of healthcare-oriented establishments = 90


Given that these numbes don't provide any context/calibration for us, let us compare them to the number of residential properties in the data set, to get an idea for scale (with the assumption that resident = consumer).

In [211]:
print("Number of food shops per residential property = {}".format(food_sum/residential_sum))
print("Number of alcohol shops per residential property = {}".format(alcohol_sum/residential_sum))
print("Number of healthcare firms per residential property = {}".format(healthcare_sum/residential_sum))

Number of food shops per residential property = 0.14366061385183995
Number of alcohol shops per residential property = 0.00948095773742568
Number of healthcare firms per residential property = 0.014462477904547646


In order to compare to the original bars per capita study referenced earlier in this report and provide ourselves with further context, we need make some assumptions and do a little more math. To begin with, we'll assume that each residential property identified is roughly equivalent to a 4-person household. As the numbers for alcohol/bars in that other study were presented "per 10,000 residents", then we'll multiply our final values by 2,500 (4 in the denominator for the residents-per-household assumption and 10,000 in the numerator to scale our results per 10,000 residents).

Also, to be fully comparable, I should really only compare with tag counts for pubs, bars, and biergartens, not all alcohol-selling shops.

This adjustment results in the following:

In [214]:
bars = ['biergarten', 'pub', 'bar']
bar_df = amenity_shop_df[amenity_shop_df['value'].isin(bars)]

bar_sum = bar_df['Num'].sum()

print("Number of 'bars' per 10,000 residents = {}".format(bar_sum/residential_sum * 2500))

Number of 'bars' per 10,000 residents = 18.881568375381647


**Interesting! This result seems to indicate that this region of WV actually maintains a higher bars per 10,000 residents count (18.9) than the booziest location identified in the 2013 study (12).** Obviously, this shouldn't be taken as a hard-and-fast result, as our assumption of 4 people per residential property could be massively off. However, it definitely points itself out as an area for future study!

# Conclusions: Thoughts on Future Directions

At this point, we've cleaned these data pretty well for our purposes and answered (at least in part) some interesting questions with it. However, there is always room for improvement! Here are some potential future directions for the wrangling and analysis that came up (largely sumamrized and collated from within the earlier narrative text):

1. **Business in the OSM data are often not identified as being of a specific type (e.g. 'shop' or 'amenity' keys).** This obviously makes analysis much more difficult and incomplete. A correction for this would be to develop a gold standard list of known businesses and extracting the business type label from that same list to put into the OSM data.

2. **Our assumption that lists of zip codes using colons are exhaustive, but more likely they are meant to be *ranges* of zip codes represented only by the beginning and ending zip code.** More GIS-based analysis would need to be done to determine if this is true and, if so, what zip codes are needed to make those truncated lists exhaustive.

3. **Many commercial locations appear to only have latitude/longitude coordinates, not street addresses, associated with them.** This makes human-readable analysis difficult. As such, it would be useful to run these coordinates through another map source (e.g. Google) to see if physical addresses can be generated from the coordinates.

4. **Our analysis of incorporated vs. unincorporated areas of the region seems flawed due to the low number of incorporated locations.** As discussed earlier, the best way to approach this is likely to generate a list of county seats (generated from an external source, like Wikipedia) from the list of counties present in this data set and do some spot checks to ensure that my logic of "county seats are always incorporated" is generally accurate and adds something useful to the analysis.

5. **Our analysis of food and healthcare deserts is a bit limited as we don't have good population count data.** In order to compare to existing studies, which are typically reported in per capita format, we would need to have more complete knowledge regarding the number of people actually living in the region, instead of having to make assumptions about the per-household resident count. Also, this analysis could be expanded in an interesting (and more useful to policy-makers) way by grouping the results by each county in the data set.